In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import umap

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# Converting labels to 1-Hot Vectors
from sklearn.preprocessing import OneHotEncoder
from mpl_toolkits.mplot3d import Axes3D


import sys
# sys.path.append("/Users/Work/Developer/interpretDL/interprettensor")
root_logdir = "./tf_logs"
datadir = "data/"
figures_dir = "data/figures/"

# To plot pretty figures
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

np.random.seed(seed=42) 
tf.__version__

Using TensorFlow backend.


'1.13.1'

In [2]:
# Helper Functions

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

######### Taken from sklearn #######
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=[8,8])
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


def get1hot(y_train,y_test):
    from sklearn.preprocessing import OneHotEncoder

    enc = OneHotEncoder(categories="auto", sparse=False)
    y_train_1hot = enc.fit_transform([[label] for label in y_train]) # Since the function expects an array of "features" per sample
    y_test_1hot = enc.fit_transform([[label] for label in y_test])

    return y_train_1hot, y_test_1hot

def get_split(features, labels):
    features = np.array(features)
    labels = np.array(labels)
    # The train set will have equal amounts of each target class
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(features, labels):
        X_train = features[train_index]
        y_train = labels[train_index]
        X_test = features[test_index]
        y_test = labels[test_index]
        
        yield X_train, y_train, X_test, y_test

def plot_history(history):
    plt.close("History")
    fig, axs = plt.subplots(1, 2, figsize=(12,6),num="History")

    # Plot training & validation accuracy values
    axs[0].grid(True)
    axs[0].plot(history.history['acc'])
    axs[0].plot(history.history['val_acc'])
    axs[0].set(title='Model accuracy', ylabel='Accuracy', xlabel='Epoch')
    axs[0].legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    axs[1].grid(True)
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set(title='Model loss',ylabel='Loss', xlabel='Epoch')
    axs[1].legend(['Train', 'Test'], loc='upper left')

    plt.show()


def remove_label(features, labels, label="MCI"):
    labels = pd.Series(fused_labels)
    non_samples = labels != label

    stripped_features = features[non_samples]
    stripped_labels = labels[non_samples]

    return stripped_features, stripped_labels

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

class AttributeRemover(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return # Doesn't do anything
    
    def transform(self, X, y=None):
        return X.drop(columns=self.attribute_names)

class OverSampler(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, random_state=42):
        self.smote = SMOTE(random_state=random_state)
    
    def fit(self, X, y=None):
        return None
    
    def transform(self, X, y=None):
        return self.smote.fit_resample(X,y)

class dfHotEncoder(BaseEstimator, TransformerMixin):
    """
    Builds a hot encoder froma pandas dataframe
    Since the function expects an array of "features" per sample,
    we reshape the values
    """
    def __init__(self, random_state=42):
        self.enc = OneHotEncoder(categories="auto", sparse=False)
        self.categories_ = None
        return None
    
    def fit(self, labels):
        self.enc.fit(labels.values.reshape(-1,1))
        self.categories_ = self.enc.categories_
        return self
    
    def transform(self, labels):
        return self.enc.transform(labels.values.reshape(-1,1))
    
# Not used
train_pipeline = Pipeline([
                    ("smote", OverSampler()),
                    ("normalizer", StandardScaler()) ])

In [4]:
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.datasets.samples_generator import make_blobs
from matplotlib import pyplot
from pandas import DataFrame

'''
Returns orginal samples, labels and modded_samples,modded_labels
'''
def modded_iris():

    from sklearn import datasets

    iris = datasets.load_iris()

    features = pd.DataFrame(iris["data"])
    target = pd.Series(iris["target"])
    flower_names = iris["target_names"]
    feature_names = iris["feature_names"]
    print(features.info())

    ### Get the first 2 flower samples

    setosa = target == 0
    versicolor = target == 1
    samples = features[setosa | versicolor]
    labels = target[setosa | versicolor]
    class_size = sum(versicolor)

    versicolor_samples = features[versicolor]
    versicolor_labels = target[versicolor]
    setosa_samples = features[setosa]

    ### Splitting *versicolor* into two sub classes

    versicolor_samples.describe()

    ## Constructing different noise sources
    gauss_noise = np.random.normal(loc=1,scale=0.25, size=[class_size//2,2])
    gauss_noise[gauss_noise < 0] = 0
    unif_noise = np.random.uniform(low=0,high=1)
    constant = 1


    split_size = class_size//2

    # Positive to first two features

    B1 = versicolor_samples.iloc[:split_size,:2] + gauss_noise
    B1 = np.concatenate((B1, versicolor_samples.iloc[:split_size,2:]), axis=1)
    B1_labels = versicolor_labels.iloc[:split_size]

    # Negative to last two features
    # gauss_noise = np.random.normal(loc=0.1,scale=0.1, size=[class_size//2,2])
    # gauss_noise[gauss_noise < 0] = 0
    # unif_noise = np.random.uniform(low=0,high=1)

    # B2 = versicolor_samples.iloc[split_size:,2:] + gauss_noise
    # B2 = np.concatenate((versicolor_samples.iloc[split_size:,2:],B2), axis=1)

    B2 = versicolor_samples.iloc[split_size:,:2] - gauss_noise
    B2 = np.concatenate((B2,versicolor_samples.iloc[split_size:,2:]), axis=1)
    B2_labels = versicolor_labels.iloc[split_size:] + 1

    # Combining the two fake "subclasses"
    noisy_samples = np.concatenate((B1, B2), axis=0)


    modded_samples = np.concatenate((setosa_samples,noisy_samples))
    modded_labels = labels.copy()
    modded_labels[class_size + split_size:] += 1

    return samples,labels,modded_samples, modded_labels


'''
Returns 8 gaussian blobs surrounding one center blob

       labels: Only center vs other labels (0,1) 
modded_labels: The labels for all 9 classes
'''
def simulate_blobs(class_size = 200):
    centers = [2*(x,y) for x in range(-1,2) for y in range(-1,2)]
    n_samples = [class_size//(len(centers)-1)]*len(centers)
    n_samples[len(centers)//2] = class_size

    X, y = make_blobs(n_samples=n_samples, centers=centers, n_features=2, cluster_std=0.1, shuffle=False, random_state=42)

    plt.close("Original Distribution")
    df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
    fig, ax = plt.subplots(num= "Original Distribution")
    colors = {0:'red', 1:'blue'}
    df.plot(ax=ax,kind="scatter", x='x', y='y',c="label", cmap= "Paired")
    # plt.colorbar()
    plt.show()
    
    original_labels = df["label"].copy()
    modded_samples = df[["x","y"]].copy()
    labels = df["label"].copy()
    labels[labels != 4] = 0
    labels[labels == 4] = 1
    return df, modded_samples,labels, original_labels

## Half the data will be split out as validation and 0.2 as the test set

In [5]:

def get_split_index(features, labels, test_size=0.1):
    features = np.array(features)
    # The train set will have equal amounts of each target class
    # Performing single split
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    return [[train_index, test_index] for train_index,test_index in split.split(features, labels)]

def split_valid(features, original_labels, training_labels):
    train_index, validation_index = get_split_index(features, original_labels, test_size=0.5)[0]
    
    X_valid, y_valid, y_valid_original = features.iloc[validation_index],  training_labels.iloc[validation_index], original_labels.iloc[validation_index]
    X_train, y_train, y_original = features.iloc[train_index], training_labels.iloc[train_index], original_labels.iloc[train_index]
     
    return X_train, y_train, y_original, X_valid, y_valid, y_valid_original

def get_train_test_val(features, original_labels, training_labels):
    
    X, y, y_original, X_valid, y_valid, y_valid_original = split_valid(features,original_labels, training_labels)
   
    train_index, test_index = get_split_index(X, y_original)[0]
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    return X_train, y_train, X_test, y_test, y_original, X_valid, y_valid, y_valid_original

### Train a DNN on the modified dataset

In [6]:
# Get split returns a generator
# List comprehension is one way to evaluate a generator

original_data, modded_samples, training_labels, original_labels = simulate_blobs(class_size=6000)

# Separating a hold out set that will be used for validation later
X_train, y_train, X_test, y_test, y_original, X_valid, y_valid, y_valid_original = get_train_test_val(modded_samples, original_labels, training_labels)


print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)


hot_encoder = dfHotEncoder()
hot_encoder.fit(training_labels)
print("Categories:", hot_encoder.categories_)

FigureCanvasNbAgg()

Train Size: (5400, 2)
Test Size: (600,)
Categories: [array([0, 1])]


In [7]:
NUM_FEATURES = X_train.shape[1]
NUM_LABELS = len(hot_encoder.categories_[0])

In [8]:
def build_dnn(num_features, num_labels=3):

#     reset_graph()
    
    keras.backend.clear_session()

    nn = keras.models.Sequential()
    Dense = keras.layers.Dense
    
    # Using He initialization
    he_init = tf.keras.initializers.he_uniform()
    
    nn.add(Dense(units = 20, activation="elu", input_dim=num_features,
                kernel_initializer=he_init))
    nn.add(Dense(units = 20, activation="elu",
                kernel_initializer=he_init))
    nn.add(Dense(units = 20, activation="elu",
                kernel_initializer=he_init))
    nn.add(Dense(units = 20, activation="elu",
            kernel_initializer=he_init))
    nn.add(Dense(units=2, activation= "softmax",
                kernel_initializer=he_init))

#     BCE = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    nn.compile(loss="categorical_crossentropy",
                  optimizer='sgd',
                  metrics=['accuracy'])
    
    return nn

def train_model(model, X, y, X_test=[], y_test=[], epochs=30, batch_size=20, verbose=1, plot=True):
    
    ZScaler = StandardScaler().fit(X)
    
    X_train = ZScaler.transform(X)
    X_test = ZScaler.transform(X_test)
    
    y_train = hot_encoder.transform(y)
    y_test = hot_encoder.transform(y_test)
    
#     lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay)
    callback_list = []
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size,
                        validation_data=(X_test, y_test), callbacks=callback_list, verbose=verbose)
    
#     if plot: plot_history(history)
    
    return history, ZScaler


In [9]:
nn = build_dnn(NUM_FEATURES)
%time history, Zscaler = train_model(nn, X_train, y_train, X_test, y_test, epochs=200, batch_size=20)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 5400 samples, validate on 600 samples
Epoch 1/200
5400/5400 [==============================] - 0s 75us/step - loss: 0.0715 - acc: 0.9896 - val_loss: 0.0155 - val_acc: 1.0000
Epoch 2/200
5400/5400 [==============================] - 0s 40us/step - loss: 0.0089 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 3/200
5400/5400 [==============================] - 0s 40us/step - loss: 0.0047 - acc: 1.0000 - val_loss: 0.0045 - val_acc: 1.0000
Epoch 4/200
5400/5400 [==============================] - 0s 39us/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 5/200
5400/5400 [==============================] - 0s 41us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0025 - val_acc: 1.0000
Epoch 6/200
5400/5400 [==============================] - 0s 38us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 7/200
5400/

In [10]:
# Plotting results from history
plot_history(history)

FigureCanvasNbAgg()

In [11]:
preds = [x for x in nn.predict(Zscaler.transform(X_test[:5]))]
_labels = [np.float(x) for x in y_test]
preds[:5],_labels[:5]

([array([2.3593466e-05, 9.9997640e-01], dtype=float32),
  array([1.000000e+00, 9.631469e-11], dtype=float32),
  array([3.0715414e-06, 9.9999690e-01], dtype=float32),
  array([9.9999893e-01, 1.0504422e-06], dtype=float32),
  array([1.0000000e+00, 1.9152106e-12], dtype=float32)],
 [1.0, 0.0, 1.0, 0.0, 0.0])

## Performing SVM on Modded Samples

In [12]:
from sklearn.svm import LinearSVC

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("SVM", LinearSVC(C=1, loss="hinge", max_iter=1000 ))
])

%time svm_clf.fit(X_train, y_train)
print("Linear SVM Test Accuracy: {:0.3f}".format(svm_clf.score(X_test, y_test)))

CPU times: user 8.72 ms, sys: 1.31 ms, total: 10 ms
Wall time: 9.04 ms
Linear SVM Test Accuracy: 0.687


## Performing LRP

In [13]:
model = nn
scaled_samples = Zscaler.transform(X_valid)
_labels = y_valid
# mod_labels = modded_labels[test_index]

predictions = model.predict(scaled_samples)
preds = np.array([np.argmax(x) for x in predictions])
true_labels = np.array([x for x in _labels])

correct = preds == true_labels
# versicolor = true_labels == 1

print("Validation Accuracy")
loss_and_metrics = model.evaluate(scaled_samples, hot_encoder.transform(y_valid))
print("Scores on validation set: loss={:0.3f} accuracy={:.4f}".format(*loss_and_metrics))

Validation Accuracy
6000/6000 [==============================] - 0s 9us/step
Scores on validation set: loss=0.000 accuracy=1.0000


In [14]:
_labels[correct].value_counts()

1    3000
0    3000
Name: label, dtype: int64

In [15]:
import innvestigate
import innvestigate.utils as iutils

def perform_analysis(model, analyzer, data, labels=[]):
    analysis = analyzer.analyze(data)
    prediction = model.predict(data)
    
    df_anal = pd.DataFrame(analysis)
    
    return df_anal


# Stripping the softmax activation from the model
model_w_softmax = nn
model = iutils.keras.graph.model_wo_softmax(model_w_softmax)

# Creating an analyzer
lrp_E = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model=model, epsilon=1e-3)
lrp_Z = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPZPlus(model=model)
lrp_AB   = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha2Beta1(model=model)

# Getting all the samples that can be correctly predicted
test_idx = correct
all_samples = scaled_samples[test_idx]
all_labels = y_valid_original[test_idx]


# perform_analysis(nn,gradient_analyzer,flowers,types)
all_lrp_AB = perform_analysis(model,lrp_AB, all_samples)
all_lrp_E = perform_analysis(model,lrp_E, all_samples)
all_lrp_Z = perform_analysis(model,lrp_Z, all_samples)


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [16]:
plt.close("Comparison")
fig, axs = plt.subplots(2,2, figsize=(16,10), num="Comparison")
cmap = "Set1" #"Paired"
plot_args = {"kind":"scatter", "x":0,  "y":1, "c":"label", "cmap": cmap, "s":10, "alpha":0.25}

original_data.plot(ax=axs[0][0],title="Original Distribution", **plot_args)

plot_args["c"] = all_labels
all_lrp_E.plot(ax=axs[0][1], title="LRP E", **plot_args)

all_lrp_AB.plot(ax=axs[1][0], title="LRP AB", **plot_args)
all_lrp_Z.plot(ax=axs[1][1], title="LRP Z", **plot_args)

plt.tight_layout()
plt.show()
# plt.savefig(figures_dir+"multiclass_lrp.png")

FigureCanvasNbAgg()

In [17]:
# import time
# plt.show(block=False)
# time.sleep(3)
# plt.close('all')

In [34]:
plt.close("Positive Only LRP")
fig, axs = plt.subplots(1,3, figsize=(18,6), num="Positive Only LRP")

plot_args["c"] = "label"
original_data.plot(ax=axs[0], title="Original Distribution", **plot_args)

plot_args["c"] = all_labels
all_lrp_E.plot(ax=axs[1], title="LRP E", **plot_args)

pos_lrp = all_lrp_E.copy()
pos_lrp[pos_lrp<0] = 0
pos_lrp["label"] = all_labels.values
pos_lrp.plot(ax=axs[2],title="LRP E", **plot_args)

plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

In [19]:
# plt.savefig(figures_dir+"deep_noisy_lrp.png")

In [35]:
color_key = {0:"red",1:"blue", 2:"green", 3:"purple", 4: "orange", 5:"yellow", 6:"brown", 7:"hotpink", 8:"grey"}
grid_pos = {0:6, 1:3, 2:0, 3:7, 4:4, 5:1, 6:8, 7:5, 8:2}
x_min,y_min,_ = pos_lrp.min()
x_max,y_max,_ = pos_lrp.max()

plt.close("Class Level LRP")
fig, axs = plt.subplots(3,3, figsize=(14,14), num="Class Level LRP")
axs = axs.flatten()
grouped = pos_lrp.groupby(by="label")
for key,group in grouped:
    group.plot(ax=axs[grid_pos[key]],kind="scatter",x=0,y=1, title=key, color=color_key[key], xlim = (x_min,x_max), ylim=(y_min,y_max), grid=True)


FigureCanvasNbAgg()

### Testing LRP outputs

In [178]:
all(all_lrp_E[[0]].values != all_lrp_E[[1]].values)

True

In [179]:
_sample = all_samples[:5]
print("Samples",_sample)
nn.predict(_sample)

Samples [[ 1.49205427 -1.50377834]
 [-0.03193957 -0.16489858]
 [-0.01627217  0.00651761]
 [ 0.03154761 -0.09189052]
 [ 1.70255546  0.17392826]]


array([[9.9999976e-01, 1.7896870e-07],
       [1.0913958e-05, 9.9998903e-01],
       [2.9683627e-06, 9.9999702e-01],
       [3.9718543e-06, 9.9999607e-01],
       [9.9999952e-01, 5.2697823e-07]], dtype=float32)

In [115]:
perform_analysis(model,lrp_Z, _sample)

,0,1
0,1.298364,2.932839
1,1.434336,1.111248
2,2.242666,1.444997
3,2.428062,1.561046
4,1.023349,0.284107


In [116]:
perform_analysis(model,lrp_E, _sample)

,0,1
0,-0.078283,0.006593
1,2.020087,2.271289
2,-0.336848,-0.911339
3,0.032239,-1.068051
4,4.087845,-0.051894


In [36]:
# pos_lrp[pos_lrp[[0,1]] < 0]